In [2]:
import os
import re
from typing import List

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import vllm
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
from numpy.linalg import norm
from peft import LoraConfig, get_peft_model
from sklearn.neighbors import NearestNeighbors
from torch import Tensor
from tqdm.autonotebook import trange
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

ModuleNotFoundError: No module named 'triton'

In [ ]:
full_df = pd.read_csv("./data/test.csv")
IS_SUBMISSION = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN"))

rows = []
for idx, row in full_df.iterrows():
    for option in ["A", "B", "C", "D"]:
        if option == row.CorrectAnswer:
            continue

        correct_answer = row[f"Answer{row.CorrectAnswer}Text"]

        query_text = f"### SubjectName: {row['SubjectName']}\n### ConstructName: {row['ConstructName']}\n### Question: {row['QuestionText']}\n### Correct Answer: {correct_answer}\n### Misconcepte Incorrect answer: {option}.{row[f'Answer{option}Text']}"
        rows.append(
            {
                "query_text": query_text,
                "QuestionId_Answer": f"{row.QuestionId}_{option}",
                "ConstructName": row.ConstructName,
                "SubjectName": row.SubjectName,
                "QuestionText": row.QuestionText,
                "correct_answer": correct_answer,
                "incorrect_answer": row[f"Answer{option}Text"],
            }
        )

df = pd.DataFrame(rows)